# 3  Missing-Data Imputation  

> “Missing values should be handled carefully to avoid biased results.” — Gotten from the OECD Handbook on Composite Indicators

It is without doubt that there will be missing data in any financial dataset just like the Russell 3000 universe which could be caused by late filings, thin traded days and IPO - Investment Blackout periods.

In this notebook  I will:

1. Diagnose the missingness pattern
2. Choose methods that balance robustness and runtime
3. Generate *five multiply-imputed* fundamental data sets (MICE)
4. Forward-fill price-series gaps
5. Save artefacts 